In [0]:
import json
import os
import mlflow
import pandas as pd
import random
from datetime import datetime, timedelta

In [0]:

# Parameters
CATALOG = "mzervou"
SCHEMA = "healthcare"
SOURCE_TABLE = f"{CATALOG}.{SCHEMA}.synthetic_audio_transcripts"
MODEL_NAME = "databricks-meta-llama-3-3-70b-instruct"

# Step 1: Load the source table
source_df = spark.table(SOURCE_TABLE)

# Step 1: SOAP Note Generation

## What we do:
Use ai_query to generate structured SOAP notes from unstructured doctor transcripts.

## Why it's important:
- Standardizes medical documentation.
- Easier for clinicians to read and verify.
- Makes future queries and analytics possible.


In [0]:
from pyspark.sql.functions import expr

df_transcripts = spark.read.table("mzervou.healthcare.synthetic_audio_transcripts")

df_soap = df_transcripts.withColumn(
    "soap_note",
    expr(
        f"""
        ai_query(
            '{MODEL_NAME}',
            concat(
                'You are a clinical AI assistant. Convert this audio transcript into a structured SOAP note (Subjective, Objective, Assessment, Plan): ',
                audio_transcript
            )
        )
        """
    )
)

df_soap.write.mode("overwrite").saveAsTable("mzervou.healthcare.synthetic_audio_transcripts_enriched")

print("✅ SOAP notes written using ai_query() to table: mzervou.healthcare.synthetic_audio_transcripts_enriched")


In [0]:
df_soap.display()

In [0]:
from pyspark.sql.functions import expr
df_transcripts = spark.read.table("mzervou.healthcare.synthetic_audio_transcripts_enriched")


# Step 2: Structured JSON Extraction

## What we do:
Use ai_query with enforced schema to extract fields like lifestyle choices like smoking, alcohol use, diet into strict JSON.
## 
## Why it's important:
- Guarantees structured, consistent data for analytics.
- Enables clean integration with BI tools and reporting.
- Supports compliance and auditability.


In [0]:
response_schema = """
{
    "type": "json_schema",
    "json_schema": {
        "name": "risk_factors",
        "schema": {
            "type": "object",
            "properties": {
                "smoking_status": { "type": "string" },
                "alcohol_use": { "type": "string" },
                "diet_quality": { "type": "string" },
                "exercise_level": { "type": "string" },
                "medication_adherence": { "type": "string" },
                "family_history_CAD": { "type": "string" }
            },
            "required": ["smoking_status", "alcohol_use", "diet_quality", "exercise_level", "medication_adherence", "family_history_CAD"]
        },
        "strict": true
    }
}
"""


In [0]:
result_structured_test = spark.sql(
    f"""
    SELECT *,
           ai_query(
               '{MODEL_NAME}',
               CONCAT(
                   'Extract structured JSON for the following risk factors from this transcript: '
                   'smoking_status, alcohol_use, diet_quality, exercise_level, medication_adherence, family_history_CAD. Transcript: ', 
                   audio_transcript
               ),
               responseFormat => '{response_schema}'
           ) AS risk_factors_structured_output
    FROM mzervou.healthcare.synthetic_audio_transcripts_enriched
    """
)

result_structured_test.display()


In [0]:
response_schema = """
{
    "type": "json_schema",
    "json_schema": {
        "name": "risk_factors",
        "schema": {
            "type": "object",
            "properties": {
                "smoking_status": { 
                    "type": "string",
                    "enum": ["None", "Current", "Former"]
                },
                "alcohol_use": { 
                    "type": "string",
                    "enum": ["None", "Light", "Moderate", "Heavy"]
                },
                "diet_quality": { 
                    "type": "string",
                    "enum": ["Poor", "Average", "Good"]
                },
                "exercise_level": { 
                    "type": "string",
                    "enum": ["None", "Low", "Moderate", "High"]
                },
                "medication_adherence": { 
                    "type": "string",
                    "enum": ["None", "Poor", "Moderate", "Good"]
                },
                "family_history_CAD": { 
                    "type": "string",
                    "enum": ["Yes", "No", "Unknown"]
                }
            },
            "required": ["smoking_status", "alcohol_use", "diet_quality", "exercise_level", "medication_adherence", "family_history_CAD"]
        },
        "strict": true
    }
}
"""


In [0]:
result_structured = spark.sql(
    f"""
    SELECT *,
           ai_query(
               '{MODEL_NAME}',
               CONCAT(
                   'Extract structured JSON for the following risk factors from this transcript: '
                   'smoking_status, alcohol_use, diet_quality, exercise_level, medication_adherence, family_history_CAD. Transcript: ', 
                   audio_transcript
               ),
               responseFormat => '{response_schema}'
           ) AS risk_factors_structured_output
    FROM mzervou.healthcare.synthetic_audio_transcripts_enriched
    """
)

result_structured.display()

# Step 3: Visit Sentiment Classification 

## What we do:
Use ai_classify to label each encounter as Routine, Urgent, or Emergency.

## Why it's important:
- Enables operational triage insights.
- Helps prioritize care or escalate reviews.
- Can flag risky cases automatically.

# Step 3+ --> Name Masking

In [0]:
from pyspark.sql.functions import col

df_enriched = (
    result_structured 
    .withColumn(
      "visit_sentiment",
      expr(
        f"""
        ai_classify(
            audio_transcript,
            array('Routine', 'Urgent', 'Emergency')
        )
        """
      )
    )
    .withColumn(
    "masked_transcript",
    expr(
        """
        ai_mask(
            audio_transcript,
            array('person', 'location')
        )
        """
    )
)
)

df_enriched.display()

# Step 4: Multilingual Translations


## What we do:
Use ai_translate to convert SOAP notes into Spanish.

### Why it's important:
- Supports international patients and providers.
- Enhances inclusivity and care quality.
- Prepares data for multi-language environments.


In [0]:
# Translation to Spanish
df_enriched_final = (
    df_enriched    
    .withColumn(
        "soap_note_spanish",
        expr(
            """
            ai_translate(
                soap_note,
                'es'
            )
            """
        )
)
)
df_enriched_final.display()

In [0]:
df_enriched_final.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("mzervou.healthcare.synthetic_audio_transcripts_enriched")
